# Preprocessing for large domain scream output

1. save crh percentiles as new file
3. binned_by_crh
4. convert omega to w vertical velocity



In [1]:
import xarray as xr
import numpy as np
import util
import proc

run="aa_default"
ndays=5
run_dir = f"/glade/derecho/scratch/sturbeville/DPSCREAM_simulations/dpscream_rce_large_3km_{run}/run/"
run_file=run_dir+f"dpscream_rce_large_3km_{run}.eam.h0.2000-01-01-00000.nc"

## 1. save the crh percentiles as a new file

In [4]:
bin_edges = np.arange(0,101,5)
crh = xr.open_dataset(run_file).isel(time=slice(-20,-1)).TMQ
crh_perc_bins = crh.quantile(bin_edges/100).values
crh_perc_bins

array([ 7.82794666, 13.98968549, 15.62247267, 16.90222931, 17.99569702,
       19.01301527, 20.01072464, 21.07153988, 22.21853333, 23.48241892,
       24.87201691, 26.38541012, 28.0511116 , 29.98027105, 32.19425392,
       34.85178185, 37.94764175, 41.07606716, 43.71133995, 45.93697548,
       64.557724  ])

In [2]:
file=run_dir+f"dpscream_rce_large_3km_{run}.eam.h0.2000-01-01-00000.nc"
proc.save_crh_percentiles(file, save2d=True)

crh < 5% i.e., 0 to 5 with a bin mid of 2.5 less than 13.989685487747192
...  140083
1 crh is between 5 and 10 with percentiles between 13 15 with a bin mid of 7.5
...  140084
2 crh is between 10 and 15 with percentiles between 15 16 with a bin mid of 12.5
...  140082
3 crh is between 15 and 20 with percentiles between 16 17 with a bin mid of 17.5
...  140084
4 crh is between 20 and 25 with percentiles between 17 19 with a bin mid of 22.5
...  140083
5 crh is between 25 and 30 with percentiles between 19 20 with a bin mid of 27.5
...  140083
6 crh is between 30 and 35 with percentiles between 20 21 with a bin mid of 32.5
...  140084
7 crh is between 35 and 40 with percentiles between 21 22 with a bin mid of 37.5
...  140082
8 crh is between 40 and 45 with percentiles between 22 23 with a bin mid of 42.5
...  140084
9 crh is between 45 and 50 with percentiles between 23 24 with a bin mid of 47.5
...  140083
10 crh is between 50 and 55 with percentiles between 24 26 with a bin mid of 52.

<xarray.Dataset> Size: 22MB
Dimensions:    (time: 19, ncol: 147456)
Coordinates:
  * time       (time) object 152B 2000-02-15 06:00:00 ... 2000-02-19 18:00:00
Dimensions without coordinates: ncol
Data variables:
    TMQ_percs  (time, ncol) float64 22MB 12.5 12.5 12.5 12.5 ... 2.5 7.5 2.5 7.5
Attributes:
    name:     column-integrated water vapor percentiles

## 2. concatenate binned_by_crh files

First, do preprocessing in preproc.sh to get the variable you want to be binned my CRH percentiles.
The CRH percentiles were generated by the ```save_crh_percentiles()``` fucntion in proc.py
Here, we use the ```cat_var_binned_by_crh()``` function to concatenate the separate files generated in the ```preproc.sh``` file

In [3]:
vars=["FSUTOA"]
for var in vars:
    file_base = f"temp_{var}_out_crh_bins_daytime"
    new_file = f"dpscream_rce_large_3km_{run}_{var.lower()}_binned_by_crhp_daytime_last5days"
    ds = proc.cat_var_binned_by_crh(var, run_dir, file_base, new_file, save2d=True)
    print("done!")


getting FSUTOA from /glade/derecho/scratch/sturbeville/DPSCREAM_simulations/dpscream_rce_large_3km_aa_default/run/temp_FSUTOA_out_crh_bins_daytime_i.nc
0 0 /glade/derecho/scratch/sturbeville/DPSCREAM_simulations/dpscream_rce_large_3km_aa_default/run/temp_FSUTOA_out_crh_bins_daytime_00.nc 162.31930541992188
1 5 /glade/derecho/scratch/sturbeville/DPSCREAM_simulations/dpscream_rce_large_3km_aa_default/run/temp_FSUTOA_out_crh_bins_daytime_05.nc 162.31930541992188
2 10 /glade/derecho/scratch/sturbeville/DPSCREAM_simulations/dpscream_rce_large_3km_aa_default/run/temp_FSUTOA_out_crh_bins_daytime_10.nc 162.31930541992188
3 15 /glade/derecho/scratch/sturbeville/DPSCREAM_simulations/dpscream_rce_large_3km_aa_default/run/temp_FSUTOA_out_crh_bins_daytime_15.nc 162.31930541992188
4 20 /glade/derecho/scratch/sturbeville/DPSCREAM_simulations/dpscream_rce_large_3km_aa_default/run/temp_FSUTOA_out_crh_bins_daytime_20.nc 162.31930541992188
5 25 /glade/derecho/scratch/sturbeville/DPSCREAM_simulations/dpsc

# 3. temp profile of end to iop file for large domain simulation

## 3. omega2w

In [ ]:
ds = xr.open_dataset(run_file, chunks={'ncol':1000})[["OMEGA","T"]].isel(time=slice(-4*ndays,-2))
ds.OMEGA

In [ ]:
w = util.omega2w(ds.OMEGA, ds.lev/100, ds['T'])
ds_w = xr.Dataset({"W":w}, attrs={'long_name':'vertical velocity',
                                  'units':'m/s'})
ds_w

In [ ]:
ds_w.to_netcdf(run_dir+f"dpscream_rce_large_3km_{run}_W_last{ndays}days.nc")